In [1]:
import os
import dask
from dask.diagnostics import ProgressBar
import numpy as np
import my_funs
import xarray as xr
import cProfile
import zarr
import matplotlib.pylab as plt
import geopandas as gpd
import folium
from folium.plugins import MousePosition

from dask.distributed import Client, progress

client = Client(processes=False)

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/"


Filtering the LAI and NDVI data based on quality control 

In [ ]:
ds_lai = xr.open_zarr(
    dir + "data/raw_data/noaa_cdr/lai_fapar/noaa_lai_fapar_clipped_raw.zarr"
)
lai = ds_lai["LAI"]
fapar = ds_lai["FAPAR"]
qa_lai = ds_lai["QA"]
ds_ndvi = xr.open_zarr(dir + "data/raw_data/noaa_cdr/ndvi/noaa_ndvi_clipped_raw.zarr")
ndvi = ds_ndvi["NDVI"]
qa_ndvi = ds_ndvi["QA"]
# Convert QAs to binary
qa_lai_binary = my_funs.dec2bin(qa_lai, 9)
qa_ndvi_binary = my_funs.dec2bin(qa_ndvi, 16)


Apply mask based on binary QAs aquired from previous step

In [ ]:
qa_lai_binary = qa_lai_binary.astype(str)
qa_lai_mask = my_funs.avhrr_mask(qa_lai_binary, dask="allowed", var="LAI")
lai_masked = lai.where(qa_lai_mask)
fapar_maked = fapar.where(qa_lai_mask)

qa_ndvi_binary = qa_ndvi_binary.astype(str)
qa_mask_ndvi = my_funs.avhrr_mask(qa_ndvi_binary, dask="allowed", var="NDVI")
ndvi_masked = ndvi.where(qa_mask_ndvi)


In [ ]:
with ProgressBar():
    lai_filtered = lai_masked.compute()
with ProgressBar():
    ndvi_filtered = ndvi_masked.compute()

# In case staritng the dask distributed client the above progressbar does not work and we should use the following commands
fapar_maked_ds = fapar_maked.to_dataset()
m = client.compute(fapar_maked_ds)
ndvi_masked_ds = ndvi_masked.to_dataset()
m2 = client.compute(ndvi_masked_ds)


In [ ]:
progress(m)
fapar_maked_ds.to_zarr(
    out_dir + "data/processed_data/noaa_nc/lai_fapar/filtered/fapar_filtered.zarr"
)


In [ ]:
progress(m2)
ndvi_masked_ds.to_zarr(
    out_dir + "data/processed_data/noaa_nc/ndvi/filtered/ndvi_filtered.zarr"
)


Resample daily LAI and NDVI to annual, growing season,seasonal and monthly

In [2]:
lai = xr.open_dataarray(
    dir + "/data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered.nc"
)

lai_monthly_resample_mean = lai.resample(time="1MS").mean()
lai_monthly_resample_mean.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_mean.nc"
)

lai_growing_monthly_resample_mean = lai_monthly_resample_mean.where(
    lai_monthly_resample_mean.time.dt.month.isin([5, 6, 7, 8, 9, 10]), drop=True
)
lai_growing_monthly_resample_mean.to_netcdf(
    dir
    + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_monthly_resample_mean.nc"
)
print("lai_monthly_resample_mean Done!")

lai_monthly_resample_max = lai.resample(time="1MS").max()
lai_monthly_resample_max.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_max.nc"
)
print("lai_monthly_resample_max Done!")

lai_monthly_group_mean = lai.groupby("time.month").mean()
lai_monthly_group_mean.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_group_mean.nc"
)
print("lai_monthly_group_mean Done!")

lai_monthly_group_max = lai.groupby("time.month").max()
lai_monthly_group_max.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_group_max.nc"
)
print("lai_monthly_group_max Done!")

lai_seasonal_resample_mean = my_funs.weighted_season_resmaple(lai)
lai_seasonal_resample_mean.to_netcdf(
    dir
    + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seasonal_resample_mean.nc"
)
print("lai_seasonal_resample_mean Done!")

lai_seaonal_group_mean = my_funs.weighted_season_group(lai)
lai_seaonal_group_mean.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seaonal_group_mean.nc"
)
print("lai_seaonal_group_mean Done!")

lai_seasonal_resample_max = lai.resample(time="Q-DEC").max()
lai_seasonal_resample_max.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seasonal_resample_max.nc"
)
print("lai_seasonal_resample_max Done!")

lai_seasonal_group_max = lai.groupby("time.season").max()
lai_seasonal_group_max.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seasonal_group_max.nc"
)
print("lai_seasonal_group_max Done!")

lai_annualresample_mean = lai.groupby("time.year").mean()
lai_annualresample_mean.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annualresample_mean.nc"
)
print("lai_annualresample_mean Done!")

lai_annualresample_max = lai.groupby("time.year").max()
lai_annualresample_max.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annualresample_max.nc"
)
print("lai_annualresample_max Done!")

lai_growing = my_funs.growing_season(lai)
lai_growing.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing.nc"
)
print("All done!")


All done!
